In [1]:
import os, glob, cv2, time
import numpy as np
import tensorflow as tf

from utils import *
from scipy import io as mat
from matplotlib import pyplot as plt

tf.set_random_seed(777)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tmp = np.load('./data/input/data_3d_minmax_after.npz')
img = tmp['img']
img = np.reshape(img, [20,186,256,256,1])
lab = tmp['lab']
lab = np.reshape(lab, [20,186,256,256,1])

In [3]:
print(img.shape)
print(lab.shape)

(20, 186, 256, 256, 1)
(20, 186, 256, 256, 1)


In [4]:
X = tf.placeholder(tf.float32, shape=[None, 186, 256, 256, 1])
Y = tf.placeholder(tf.float32, shape=[None, 186, 256, 256, 1])

In [5]:
with tf.variable_scope('variable'):
    w = {
        'wen1':tf.get_variable('wen1', shape=[3,3,3,1,32],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'wen2':tf.get_variable('wen2', shape=[7,7,7,32,256],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'wen3':tf.get_variable('wen3', shape=[5,5,5,256,512],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'wen4':tf.get_variable('wen4', shape=[3,3,3,512,512],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'wen5':tf.get_variable('wen5', shape=[3,3,3,512,512],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'wen6':tf.get_variable('wen6', shape=[3,3,3,512,512],
                          initializer=tf.contrib.layers.xavier_initializer()),
                
        
        'wde6':tf.get_variable('wde6', shape=[4,4,4,512,512],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'wde5':tf.get_variable('wde5', shape=[4,4,4,512,512],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'wde4':tf.get_variable('wde4', shape=[4,4,4,512,512],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'wde3':tf.get_variable('wde3', shape=[4,4,4,256,512],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'wde2':tf.get_variable('wde2', shape=[4,4,4,128,256],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'wde1':tf.get_variable('wde1', shape=[4,4,4,1,128],
                          initializer=tf.contrib.layers.xavier_initializer()),
    }
    b = {
        'ben1':tf.get_variable('ben1', shape=[32],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'ben2':tf.get_variable('ben2', shape=[256],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'ben3':tf.get_variable('ben3', shape=[512],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'ben4':tf.get_variable('ben4', shape=[512],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'ben5':tf.get_variable('ben5', shape=[512],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'ben6':tf.get_variable('ben6', shape=[512],
                          initializer=tf.contrib.layers.xavier_initializer()),
        
        'bde6':tf.get_variable('bde6', shape=[512],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'bde5':tf.get_variable('bde5', shape=[512],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'bde4':tf.get_variable('bde4', shape=[512],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'bde3':tf.get_variable('bde3', shape=[256],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'bde2':tf.get_variable('bde2', shape=[128],
                          initializer=tf.contrib.layers.xavier_initializer()),
        'bde1':tf.get_variable('bde1', shape=[1],
                          initializer=tf.contrib.layers.xavier_initializer())
    }

In [6]:
def model(X, w, b):
    
    en1 = tf.nn.conv3d(X, w['wen1'], strides=[1,2,2,2,1], padding='SAME')
    en1 = tf.nn.bias_add(en1, b['ben1'])
    en1 = tf.nn.sigmoid(en1) # 93 x 128 x 128
    print(en1)
    
    en2 = tf.nn.conv3d(en1, w['wen2'], strides=[1,2,2,2,1], padding='SAME')
    en2 = tf.nn.bias_add(en2, b['ben2'])
    en2 = tf.nn.relu(en2) # 47 x 64 x 64
    print(en2)
    
    en3 = tf.nn.conv3d(en2, w['wen3'], strides=[1,2,2,2,1], padding='SAME')
    en3 = tf.nn.bias_add(en3, b['ben3'])
    en3 = tf.nn.relu(en3) # 24 x 32 x 32
    print(en3)
    
    en4 = tf.nn.conv3d(en3, w['wen4'], strides=[1,2,2,2,1], padding='SAME')
    en4 = tf.nn.bias_add(en4, b['ben4'])
    en4 = tf.nn.relu(en4) # 12 x 16 x 16
    print(en4)
    
    en5 = tf.nn.conv3d(en4, w['wen5'], strides=[1,2,2,2,1], padding='SAME')
    en5 = tf.nn.bias_add(en5, b['ben5'])
    en5 = tf.nn.relu(en5) # 6 x 8 x 8
    print(en5)
    
    en6 = tf.nn.conv3d(en5, w['wen6'], strides=[1,2,2,2,1], padding='SAME')
    en6 = tf.nn.bias_add(en6, b['ben6'])
    en6 = tf.nn.relu(en6) # 3 x 4 x 4
    print(en6)
    
    de6 = tf.nn.conv3d_transpose(en6, w['wde6'],(tf.shape(X)[0], 6, 8, 8, 512),
                                strides=[1,2,2,2,1], padding="SAME")
    de6 = tf.nn.bias_add(de6, b['bde6'])
    de6 = tf.nn.relu(de6)
    print(de6)
    
    de5 = tf.nn.conv3d_transpose(de6, w['wde5'],(tf.shape(X)[0], 12, 16, 16, 512),
                                strides=[1,2,2,2,1], padding="SAME")
    de5 = tf.nn.bias_add(de5, b['bde5'])
    de5 = tf.nn.relu(de5)
    print(de5)
    
    de4 = tf.nn.conv3d_transpose(de5, w['wde4'],(tf.shape(X)[0], 24, 32, 32, 512),
                                strides=[1,2,2,2,1], padding="SAME")
    de4 = tf.nn.bias_add(de4, b['bde4'])
    de4 = tf.nn.relu(de4)
    print(de4)
    
    de3 = tf.nn.conv3d_transpose(de4, w['wde3'],(tf.shape(X)[0], 47, 64, 64, 256),
                                strides=[1,2,2,2,1], padding="SAME")
    de3 = tf.nn.bias_add(de3, b['bde3'])
    de3 = tf.nn.relu(de3)
    print(de3)
    
    de2 = tf.nn.conv3d_transpose(de3, w['wde2'],(tf.shape(X)[0], 93, 128, 128, 128),
                                strides=[1,2,2,2,1], padding="SAME")
    de2 = tf.nn.bias_add(de2, b['bde2'])
    de2 = tf.nn.relu(de2)
    print(de2)
    
    de1 = tf.nn.conv3d_transpose(de2, w['wde1'],(tf.shape(X)[0], 186, 256, 256, 1),
                                strides=[1,2,2,2,1], padding="SAME")
    de1 = tf.nn.bias_add(de1, b['bde1'])
    de1 = tf.nn.sigmoid(de1)
    print(de1)
    
    out = {
        'en1' : en1,
        'en2' : en2,
        'en3' : en3,
        'en4' : en3,
        'en5' : en3,
        'en6' : en3,
        
        'de6' : de3,
        'de5' : de3,
        'de4' : de3,
        'de3' : de3,
        'de2' : de2,
        'de1' : de1
    }
    
    return out

In [7]:
def dice_coefficient(label, infer):
    
    label_flatten = tf.layers.flatten(label)
    infer_flatten = tf.layers.flatten(infer)
    TP = tf.reduce_sum(label_flatten*infer_flatten)
    FN = tf.reduce_sum(label_flatten*(1-infer_flatten))
    FP = tf.reduce_sum((1-label_flatten)*infer_flatten)
    
    bottom = FN+FP+(2*TP)
    up = 2*TP
    
    return up/bottom

In [8]:
with tf.device('/gpu:0'):
    pred = model(X, w, b)
    coeff = dice_coefficient(Y, pred['de1'])
    cost = 1.0-coeff
with tf.device('/gpu:1'):
    train = tf.train.AdamOptimizer(1e-7).minimize(cost)

Tensor("Sigmoid:0", shape=(?, 93, 128, 128, 32), dtype=float32, device=/device:GPU:0)
Tensor("Relu:0", shape=(?, 47, 64, 64, 256), dtype=float32, device=/device:GPU:0)
Tensor("Relu_1:0", shape=(?, 24, 32, 32, 512), dtype=float32, device=/device:GPU:0)
Tensor("Relu_2:0", shape=(?, 12, 16, 16, 512), dtype=float32, device=/device:GPU:0)
Tensor("Relu_3:0", shape=(?, 6, 8, 8, 512), dtype=float32, device=/device:GPU:0)
Tensor("Relu_4:0", shape=(?, 3, 4, 4, 512), dtype=float32, device=/device:GPU:0)
Tensor("Relu_5:0", shape=(?, 6, 8, 8, 512), dtype=float32, device=/device:GPU:0)
Tensor("Relu_6:0", shape=(?, 12, 16, 16, 512), dtype=float32, device=/device:GPU:0)
Tensor("Relu_7:0", shape=(?, 24, 32, 32, 512), dtype=float32, device=/device:GPU:0)
Tensor("Relu_8:0", shape=(?, 47, 64, 64, 256), dtype=float32, device=/device:GPU:0)
Tensor("Relu_9:0", shape=(?, 93, 128, 128, 128), dtype=float32, device=/device:GPU:0)
Tensor("Sigmoid_1:0", shape=(?, 186, 256, 256, 1), dtype=float32, device=/device:GP

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [10]:
start_time = time.time()

epochs = 10

batch = 2

total_batch = int(len(img)/batch)

for epoch in range(epochs):
    
    for i in range(total_batch):
        
        batch_idx = np.random.choice(len(img), batch, replace=False)
        
        batch_x = img[batch_idx]
        batch_y = lab[batch_idx]
        
        sess.run(train, feed_dict={X:batch_x, Y:batch_y})
        
        if (i+1)%(total_batch/2) == 0:
                        
            print("%d/%d"%(epoch+1, epochs))
            
            print("%d/%d"%(i+1, total_batch))
            
            print("Train cost : ", sess.run(cost, 
                                            feed_dict={X:batch_x, Y:batch_y}))
            print("Train coef : ", sess.run(coeff, 
                                            feed_dict={X:batch_x, Y:batch_y}))
            
            print('-----------------')
            
print('Time : ', time.time()-start_time)            

1/10
5/10
Train cost :  0.9995075
Train coef :  0.00049248716
-----------------
1/10
10/10
Train cost :  0.9988724
Train coef :  0.0011275968
-----------------
2/10
5/10
Train cost :  0.99923056
Train coef :  0.00076940673
-----------------
2/10
10/10
Train cost :  0.9992228
Train coef :  0.00077719754
-----------------
3/10
5/10
Train cost :  0.99879545
Train coef :  0.0012045732
-----------------
3/10
10/10
Train cost :  0.99818337
Train coef :  0.0018166461
-----------------
4/10
5/10
Train cost :  0.998924
Train coef :  0.0010760068
-----------------
4/10
10/10
Train cost :  0.99930966
Train coef :  0.00069035246
-----------------
5/10
5/10
Train cost :  0.99994415
Train coef :  5.58461e-05
-----------------
5/10
10/10
Train cost :  0.99906224
Train coef :  0.00093774375
-----------------
6/10
5/10
Train cost :  0.9989897
Train coef :  0.0010102942
-----------------
6/10
10/10
Train cost :  0.99862707
Train coef :  0.00137292
-----------------
7/10
5/10
Train cost :  0.99831474
Tra

In [ ]:
num = np.random.randint(0, 20) #plot_3d(np.round(tr_data[num]))
plot_3d(lab[num][:,:,:,0])
plot_3d(np.round(sess.run(pred, feed_dict={X:np.reshape(img[num], [1,186,256,256,1])})['de1'][0])[:,:,:,0])


In [ ]:
sess.run(pred, feed_dict={X:np.reshape(img[num], [1,186,256,256,1])})['de1']